In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

In [4]:
# 输入文件名输入框，默认 default.xls
file_input = widgets.Text(
    value='',
    description='Excel 文件:',
    placeholder='留空默认 default.xlsx',
    layout=widgets.Layout(width='50%')
)

load_button = widgets.Button(description="加载并排序", button_style='success')
output = widgets.Output()

display(file_input, load_button, output)

Text(value='', description='Excel 文件:', layout=Layout(width='50%'), placeholder='留空默认 default.xlsx')

Button(button_style='success', description='加载并排序', style=ButtonStyle())

Output()

In [ ]:
# 点击按钮后处理逻辑
def load_and_sort_file(b):
    with output:
        clear_output()
        
        filename = file_input.value.strip()
        if not filename:
            filename = 'default.xlsx'
        
        if not os.path.exists(filename):
            print(f"❌ 文件不存在：{filename}，请重新输入。")
            return
        
        print(f"📥 读取文件：{filename}")
        try:
            df = pd.read_excel(filename, engine='openpyxl')

            # 提取时间部分
            df['上车时间'] = df['上车地点'].str.extract(r'(\d{2}:\d{2})')
            df['上车时间'] = pd.to_datetime(df['上车时间'], format='%H:%M')

            df['座位号'] = pd.to_numeric(df['座位号'], errors='coerce')

            df_sorted = df.sort_values(by=['上车时间', '座位号'], ascending=[True, True])
            df_sorted = df_sorted.drop(columns=['上车时间'])

            # 输出文件名：原文件名 + '-领队点名版.xlsx'
            base_name = os.path.splitext(filename)[0]
            output_file = f"{base_name}-领队点名版.xlsx"
            
            df_sorted.to_excel(output_file, index=False)
            print(f"✅ 排序成功，输出文件：{output_file}")
        except Exception as e:
            print("❌ 处理出错：", e)

load_button.on_click(load_and_sort_file)